In [52]:
#import warnings
#warnings.filterwarnings("ignore")
%pip install nltk shap
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import logging
# !pip install nltk

nltk.download('vader_lexicon')

# Inicjalizacja VADER
sia = SentimentIntensityAnalyzer()

# Funkcja do analizy sentymentu
def analyze_sentiment(sentence):
    result = sia.polarity_scores(sentence)
    return result['compound'], result['neg'], result['neu'], result['pos']

df = pd.read_csv('preprocessed.csv')
print(df.head())


           id                                            content  \
0  1698308935  sure tune watch donald trump late night david ...   
1  1701461182  donald trump appearing view tomorrow morning d...   
2  1737479987  donald trump reads top ten financial tips late...   
3  1741160716  new blog post celebrity apprentice finale less...   
4  1773561338  persona never wallflower rather build walls cl...   

                  date  retweets  favorites  year  month  dayofweek  
0  2009-05-04 13:54:25       510        917  2009      5          0  
1  2009-05-04 20:00:10        34        267  2009      5          0  
2  2009-05-08 08:38:08        13         19  2009      5          4  
3  2009-05-08 15:40:15        11         26  2009      5          4  
4  2009-05-12 09:07:28      1375       1945  2009      5          1  


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\JAKUB\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [53]:
df = df.dropna(subset=['content'])
df['content'] = df['content'].astype(str)

# Analiza sentymentu dla każdego tweeta
df['compound'], df['negative'], df['neutral'], df['positive'] = zip(*df['content'].apply(analyze_sentiment))
df

,id,content,date,retweets,favorites,year,month,dayofweek,compound,negative,neutral,positive
0,1698308935,sure tune watch donald trump late night david ...,2009-05-04 13:54:25,510,917,2009,5,0,0.4767,0.00,0.745,0.255
1,1701461182,donald trump appearing view tomorrow morning d...,2009-05-04 20:00:10,34,267,2009,5,0,0.7506,0.00,0.652,0.348
2,1737479987,donald trump reads top ten financial tips late...,2009-05-08 08:38:08,13,19,2009,5,4,0.5719,0.00,0.680,0.320
3,1741160716,new blog post celebrity apprentice finale less...,2009-05-08 15:40:15,11,26,2009,5,4,0.0000,0.00,1.000,0.000
4,1773561338,persona never wallflower rather build walls cl...,2009-05-12 09:07:28,1375,1945,2009,5,1,0.0000,0.00,1.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
43347,1273405198698975232,joe biden total failure government bungled eve...,2020-06-17 19:00:32,23402,116377,2020,6,2,-0.5106,0.32,0.680,0.000
43348,1273408026968457216,interviewed seanhannity tonight 9 00 p enjoy,2020-06-17 19:11:47,11810,56659,2020,6,2,0.4939,0.00,0.556,0.444
43349,1273442195161387008,pic twitter com 3lm1spbu8x,2020-06-17 21:27:33,4959,19344,2020,6,2,0.0000,0.00,1.000,0.000
43350,1273442469066276864,pic twitter com vpce5maduz,2020-06-17 21:28:38,4627,17022,2020,6,2,0.0000,0.00,1.000,0.000


In [54]:
def categorize_sentiment(compound):
    if compound >= 0.05:
        return 'positive'
    elif compound <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df['sentiment'] = df['compound'].apply(categorize_sentiment)

df['number'] = [i+1 for i, _ in enumerate(df.iterrows())]
df = df[['number'] + list(df.columns[:-1])]

df = df.drop(columns=['id', 'retweets', 'favorites', 'year', 'month', 'dayofweek'])
df.to_csv('tweets_sentiment.csv', index=False)
df



,number,content,date,compound,negative,neutral,positive,sentiment
0,1,sure tune watch donald trump late night david ...,2009-05-04 13:54:25,0.4767,0.00,0.745,0.255,positive
1,2,donald trump appearing view tomorrow morning d...,2009-05-04 20:00:10,0.7506,0.00,0.652,0.348,positive
2,3,donald trump reads top ten financial tips late...,2009-05-08 08:38:08,0.5719,0.00,0.680,0.320,positive
3,4,new blog post celebrity apprentice finale less...,2009-05-08 15:40:15,0.0000,0.00,1.000,0.000,neutral
4,5,persona never wallflower rather build walls cl...,2009-05-12 09:07:28,0.0000,0.00,1.000,0.000,neutral
...,...,...,...,...,...,...,...,...
43347,43248,joe biden total failure government bungled eve...,2020-06-17 19:00:32,-0.5106,0.32,0.680,0.000,negative
43348,43249,interviewed seanhannity tonight 9 00 p enjoy,2020-06-17 19:11:47,0.4939,0.00,0.556,0.444,positive
43349,43250,pic twitter com 3lm1spbu8x,2020-06-17 21:27:33,0.0000,0.00,1.000,0.000,neutral
43350,43251,pic twitter com vpce5maduz,2020-06-17 21:28:38,0.0000,0.00,1.000,0.000,neutral
